In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.linear_model import *
import itertools

from keras.utils.np_utils import to_categorical 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator
import os

import xgboost as xgb 
from xgboost import plot_importance , XGBClassifier, DMatrix
import joblib


In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data_file/train.csv')
test = pd.read_csv('./drive/My Drive/DACON/data_file/test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
rot_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=45, 
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     
    zoom_range=0,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,            
    horizontal_flip=False, 
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

trans_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.2, 
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0,     
    zoom_range=0,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=False, 
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

shear_zoom_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,     
    zoom_range=0.2,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=False,
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

flip_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     
    zoom_range=0,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=True, 
    vertical_flip=True,   
    rescale=1./255, # Rescale
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

In [ ]:
def augmentation( input_imgs, aug_size ):
    df = input_imgs
    new_data_set = []
    num_of_training_set = df.shape[0]

    for i in range(num_of_training_set//2):
        rand_1 = np.random.randint(num_of_training_set)
        rand_2 = np.random.randint(num_of_training_set)
        rand_3 = np.random.randint(num_of_training_set)
        rand_4 = np.random.randint(num_of_training_set)
    
        for j in range( aug_size ):
            # rotation
            _rot = rot_gen.flow( np.array(df.iloc[rand_1,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_1,1],
                df.iloc[rand_1,2],
            ] + list(_rot)]
            # translation
            _trans = trans_gen.flow( np.array(df.iloc[rand_2,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_2,1],
                df.iloc[rand_2,2],
            ] + list(_trans)]
            # shear / zoom
            _shear = shear_zoom_gen.flow( np.array(df.iloc[rand_3,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_3,1],
                df.iloc[rand_3,2],
            ] + list(_shear)]
            # flip
            _flip = flip_gen.flow( np.array(df.iloc[rand_4,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_4,1],
                df.iloc[rand_4,2],
            ] + list(_flip)]

    columns = ['digit', 'letter'] + [str(x) for x in range(784)]
    aug = pd.DataFrame(new_data_set, columns=columns)

    train_norm = pd.concat([ input_imgs.iloc[:,1:3], np.divide(input_imgs.iloc[:,3:],255) ],axis=1)
    train_aug = pd.concat([train_norm,aug])

    return train_aug


In [ ]:
def train_test_gen(input_imgs, aug_size):
    train_aug = augmentation(input_imgs, aug_size)

    x_train = train_aug.iloc[:,2:].values.copy()
    x_train = x_train.reshape(-1,28,28,1)

    y_train = train_aug['digit']
    y_train = to_categorical(y_train,num_classes = 10)

    return train_test_split(x_train,y_train,test_size=0.1,random_state=15)

In [ ]:
def load_best(file_name):
    filepath = './drive/My Drive/DACON/saved_model/' + file_name + '/'
    time_list = []
    for f_name in os.listdir(f"{filepath}"):
        written_time = os.path.getctime(f"{filepath}{f_name}")
        time_list.append((f_name, written_time))
    sorted_file_list = sorted(time_list, key=lambda x: x[1], reverse=True)
    best = sorted_file_list[0]
    best_name = best[0]
    model = load_model( filepath + best_name )
    print('\033[31m' + best_name + '\033[0m')
    print()
    return model

In [ ]:
def set_filepath(file_name):
    MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/saved_model/' + file_name + '/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)
    
    return MODEL_SAVE_FOLDER_PATH

In [ ]:
def get_model(N):
    
    model = Sequential()

    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu', input_shape = (28,28,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = N, kernel_size = (5,5),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
                
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 2*N, kernel_size = (3,3),padding = 'Same', 
                    activation ='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(4*N, activation = "relu", name='my_dense'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))

    return model

In [ ]:
def compare(file1,file2):
    filepath1 = './drive/My Drive/DACON/submission/' + file1 +'.csv'
    filepath2 = './drive/My Drive/DACON/submission/' + file2 +'.csv'
    f1 = pd.read_csv(filepath1)
    f2 = pd.read_csv(filepath2)
    match = np.array( [ f1['digit']==f2['digit'] ][0] )
    acc = len( np.where(match==True)[0] )/len(match)
    return acc

In [ ]:
def pred_acc(file_name,file_list):
    score = []
    for i in range( len(file_list) ):
        acc = compare(file_name, file_list[i])
        score.append(acc)
        print( 'Compared with ' + file_list[i].replace('submision_','') + ' : {}'.format(acc) )
    #return score
    

In [ ]:
epochs = 100
batch_size = 100

In [ ]:
def train_cnn(cnn_num, N, aug_size):
    cnn_model_list = []
    acc_list = []

    for i in range(cnn_num):

        model = get_model(N)
        file_name = 'model_ensembles'
        model_path = set_filepath(file_name) + 'cnn_{}'.format(i) + '_{val_accuracy:.4f}.hdf5'

        # callbacks
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max')
        mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')
        # compile
        optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

        # fit model
        x_train, x_val, y_train, y_val = train_test_gen(train_copy, aug_size)

        hist = model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, 
                    validation_data = (x_val,y_val),
                    steps_per_epoch=x_train.shape[0]// batch_size, 
                    callbacks=[early_stopping,mcp_save,reduce_lr_loss])
        model = load_best(file_name)

        cnn_model_list.append(model)
        acc_list.append(hist.history['val_accuracy'][-11])

    return cnn_model_list, acc_list


In [ ]:
def train_xgb(xgb_num, N, aug_size):
    inter_model_list = []
    layer_name='my_dense'

    for i in range(xgb_num):

        model = get_model(N)
        file_name = 'model_ensembles'
        model_path = set_filepath(file_name) + 'xgb_{}'.format(i) + '_{val_accuracy:.4f}.hdf5'

        # callbacks
        early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, mode='max')
        mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')

        optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

        # fit model
        x_train, x_val, y_train, y_val = train_test_gen(train_copy, aug_size)

        hist = model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, 
                    validation_data = (x_val,y_val),
                    steps_per_epoch=x_train.shape[0]// batch_size, 
                    callbacks=[early_stopping,mcp_save,reduce_lr_loss])
        model = load_best(file_name)
        
        inter_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
        inter_model_list.append(inter_model)

    train_aug = augmentation(train_copy, aug_size)
    inter_train = train_aug.iloc[:,2:].values.reshape(-1,28,28,1)
    cnn_output_list = []
    cnn_val_list = []
    for i in range(xgb_num):
        cnn_output = inter_model_list[i].predict( inter_train ) 
        cnn_output = pd.DataFrame( data=cnn_output )
        cnn_output_list.append(cnn_output)
        cnn_val = train_aug['digit']
        cnn_val_list.append(cnn_val)

    xgb_model_list = []
    for i in range(xgb_num):
        x_train, x_val, y_train, y_val = train_test_split(cnn_output_list[i], cnn_val_list[i],test_size=0.1,random_state=25)

        xgb_model = XGBClassifier(max_depth=5, num_class=10, objective='multi:softprob', booster='gbtree', n_estimators=300, learning_rate=0.2 )
        xgb_model.fit( x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='mlogloss', early_stopping_rounds=5)

        xgb_model_list.append(xgb_model)

    return xgb_model_list, inter_model_list

In [ ]:
cnn_num = 10
xgb_num = 5

In [ ]:
cnn_model_list, acc_list = train_cnn(cnn_num, 64, 2)

In [ ]:
xgb_model_list, inter_model_list = train_xgb(xgb_num,128,3)

In [ ]:
def ensemble(input_imgs,cnn_model_list, inter_model_list, xgb_model_list, w_cnn, w_xgb):
    pred = []
    L = input_imgs.shape[0]
    label_list = np.zeros((L,10))
    for i in range(len(cnn_model_list)):
        label = cnn_model_list[i].predict( np.array(input_imgs).reshape(-1,28,28,1).astype(np.float32) )
        label_list += label*w_cnn

    for i in range(len(xgb_model_list)):
        cnn_output = inter_model_list[i].predict( np.array(input_imgs).reshape(-1,28,28,1).astype(np.float32) )
        cnn_output = DataFrame(cnn_output)
        label = xgb_model_list[i].predict_proba( cnn_output )
        label_list += label*w_xgb
        
    for j in range(len(label_list)):
        pred.append( np.argmax(label_list[j]) )

    return pred

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
pred = ensemble(x_test, cnn_model_list, inter_model_list, xgb_model_list)
data = {'id':test_copy['id'], 'digit':pred}
submission = DataFrame(data)
file_name = 'submission_ensembles_6+2_bn_07'
submission.to_csv('./drive/My Drive/DACON/submission/'+file_name+'.csv', index=False)
print(pred[:10])
print()
file_list = [ 'submission_84',
             'submission_85',
             'submission_86_xgb_ensemble',
             'submission_87_ensembles',
             'submission_88_ensemble_2_2_4_try3',
             'submission_89_ensemble_2_2',
             'submission_89',
             'submission_91_ensembles_3+1_w1',
             'submission_91_ensembles_6+2_bn_08'
             ]
pred_acc(file_name,file_list)

In [ ]:
def to_data(input, cnn_model_list, inter_model_list, xgb_model_list):

    pred_data = []
    a = len(cnn_model_list)
    b = len(xgb_model_list)
    for i in range(a):
           pred_data.append( cnn_model_list[i].predict(input) )
    for i in range(b):
        cnn_output = inter_model_list[i].predict( input )
        cnn_output = DataFrame(cnn_output)
        pred_data.append( xgb_model_list[i].predict_proba( cnn_output ) )

    data = pred_data[0]
    for i in range(1,a+b):
        data = np.concatenate((data, pred_data[i]),axis=1)
    
    return data

In [ ]:
def linear_reg(aug_size, cnn_model_list, inter_model_list, xgb_model_list):
    
    train_aug = augmentation(train_copy, aug_size)
    input = train_aug.iloc[:,2:].values.copy().reshape(-1,28,28,1)
    data = to_data(input, cnn_model_list, inter_model_list, xgb_model_list)
    data_val = train_aug['digit']
    data_val = to_categorical(data_val, 10)
    x_train, x_val, y_train, y_val = train_test_split(data, data_val, test_size = 0.1)
    linear_model = LinearRegression()
    linear_model.fit(x_train, y_train)
    print('Coefficients: {}', format(linear_model.coef_[0]))
    print("RSS: {}".format( np.mean((linear_model.predict(x_val) - y_val) ** 2) ))

    return linear_model

In [ ]:
train_aug_check = augmentation(train_copy,3)
x_check = train_aug_check.iloc[:,2:].values
x_check = x_check.reshape(-1,28,28,1)
y_check = train_aug_check['digit']

data_check = to_data(x_check, cnn_model_list, inter_model_list, xgb_model_list)

linear_model = linear_reg(3, cnn_model_list, inter_model_list, xgb_model_list)
pred = linear_model.predict( data_check )
pred[:10]
pred_check = []
for i in range(len(pred)):
    pred_check.append( np.argmax(pred[i]) )

s = [np.array(pred_check)==np.array(y_check)]
t = np.where(s[0]==True)
acc = len(t[0])/len(s[0])
acc

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
data_test = to_data(x_test, cnn_model_list, inter_model_list, xgb_model_list)
#linear_model = linear_reg(3, cnn_model_list, inter_model_list, xgb_model_list)

pred = linear_model.predict( data_test )
pred_test = []
for i in range(len(pred)):
    pred_test.append( np.argmax(pred[i]) )
pred_test[:10]

In [ ]:
data = {'id':test_copy['id'], 'digit':pred_test}
submission = DataFrame(data)
file_name = 'submission_ensembles_10+5_bn_linearreg'
submission.to_csv('./drive/My Drive/DACON/submission/'+file_name+'.csv', index=False)
file_list = [ 'submission_84',
             'submission_85',
             'submission_86_xgb_ensemble',
             'submission_87_ensembles',
             'submission_88_ensemble_2_2_4_try3',
             'submission_89_ensemble_2_2',
             'submission_89',
             'submission_90_ensembles_6+2_bn_08_retry',
             'submission_90_pretrain_using_test_layer_4_3ensemble',
             'submission_90_ensembles_6+2_bn_linearreg',
             'submission_91_ensembles_3+1_w1',
             'submission_91_ensembles_6+2_bn_08'
             ]
pred_acc(file_name,file_list)

In [ ]:
MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/saved_model/' + 'model_ensembles_10+5' + '/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
for i in range(len(cnn_model_list)):
    cnn_model_list[i].save(MODEL_SAVE_FOLDER_PATH  + 'cnn_model_list_{}.hdf5'.format(i))
for i in range(len(inter_model_list)):
    inter_model_list[i].save(MODEL_SAVE_FOLDER_PATH  + 'inter_model_list_{}.hdf5'.format(i))
for i in range(len(xgb_model_list)):
    joblib.dump(xgb_model_list[i], MODEL_SAVE_FOLDER_PATH  + 'xgb_model_list_{}.hdf5'.format(i))

In [ ]:
for i in range(len(xgb_model_list)):
    joblib.dump(xgb_model_list[i], MODEL_SAVE_FOLDER_PATH  + 'xgb_model_list_{}.dat'.format(i))

In [ ]:
filepath = './drive/My Drive/DACON/saved_model/model_ensembles_10+5/'
time_list = []
for f_name in os.listdir(f"{filepath}"):
    written_time = os.path.getctime(f"{filepath}{f_name}")
    time_list.append((f_name, written_time))
sorted_file_list = sorted(time_list, key=lambda x: x[1], reverse=False)
models = sorted_file_list
cnn_model_list_load = []
inter_model_list_load = []
xgb_model_list_load = []
for i in range(cnn_num):
    model = load_model( filepath + models[i][0] )
    cnn_model_list_load.append(model)

In [ ]:
for i in range(xgb_num):
    model = load_model( filepath + models[i+cnn_num][0] )
    inter_model_list_load.append(model)

In [ ]:
for i in range(xgb_num):
    model = joblib.load( filepath + models[i+cnn_num+xgb_num][0] )
    xgb_model_list_load.append(model)

In [ ]:
cnn_model_list_load[0] == cnn_model_list[0]

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
data_test_load = to_data(x_test, cnn_model_list_load, inter_model_list_load, xgb_model_list_load)
#linear_model = linear_reg(3, cnn_model_list, inter_model_list, xgb_model_list)

pred_load = linear_model.predict( data_test_load )
pred_test_load = []
for i in range(len(pred_load)):
    pred_test_load.append( np.argmax(pred_load[i]) )
pred_test_load[:10]

# Load saved models

In [ ]:
filepath = './drive/My Drive/DACON/saved_model/model_ensembles_6+2/'
time_list = []
for f_name in os.listdir(f"{filepath}"):
    written_time = os.path.getctime(f"{filepath}{f_name}")
    time_list.append((f_name, written_time))
sorted_file_list = sorted(time_list, key=lambda x: x[1], reverse=False)
models = sorted_file_list

In [ ]:
model_list = []
for i in range(len(models)):
    model = load_model( filepath + models[i][0] )
    model_list.append(model)

In [ ]:
inter_model_list = []
for i in range(2):
    model = model_list[i+6]
    inter_model = Model(inputs=model.input, outputs=model.get_layer('my_dense').output)
    inter_model_list.append(inter_model)

train_aug = augmentation(train_copy, 3)
inter_train = train_aug.iloc[:,2:].values.reshape(-1,28,28,1)
cnn_output_list = []
cnn_val_list = []
for i in range(2):
    cnn_output = inter_model_list[i].predict( inter_train ) 
    cnn_output = pd.DataFrame( data=cnn_output )
    cnn_output_list.append(cnn_output)
    cnn_val = train_aug['digit']
    cnn_val_list.append(cnn_val)

xgb_model_list = []
for i in range(2):
    x_train, x_val, y_train, y_val = train_test_split(cnn_output_list[i], cnn_val_list[i],test_size=0.1,random_state=25)

    xgb_model = XGBClassifier(max_depth=5, num_class=10, objective='multi:softprob', booster='gbtree', n_estimators=300, learning_rate=0.2 )
    xgb_model.fit( x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='mlogloss', early_stopping_rounds=5)

    xgb_model_list.append(xgb_model)

In [ ]:
cnn_model_list = model_list[:6]

In [ ]:
train_aug = augmentation(train_copy,2)
x_train = train_aug.iloc[:,2:].values.reshape(-1,28,28,1)
y_train = train_aug['digit']

In [ ]:
pred = ensemble(x_train, cnn_model_list, inter_model_list, xgb_model_list, 1, 3.8)
s = [np.array(pred)==np.array(y_train)]
t = np.where(s[0]==True)
acc = len(t[0])/len(s[0])
acc

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
pred = ensemble(x_test, cnn_model_list, inter_model_list, xgb_model_list, 1.0, 3.85)
data = {'id':test_copy['id'], 'digit':pred}
submission = DataFrame(data)
file_name = 'submission_ensembles_6+2_bn_08_385'
submission.to_csv('./drive/My Drive/DACON/submission/'+file_name+'.csv', index=False)
print(pred[:10])
print()
file_list = [ 'submission_84',
             'submission_85',
             'submission_86_xgb_ensemble',
             'submission_87_ensembles',
             'submission_88_ensemble_2_2_4_try3',
             'submission_89_ensemble_2_2',
             'submission_89',
             'submission_91_ensembles_3+1_w1',
             'submission_91_ensembles_6+2_bn_08'
             ]
pred_acc(file_name,file_list)